In [1]:
import pynq
import numpy as np

In [2]:
# program the device
ol = pynq.Overlay("vitis_xclbin/vadd.xclbin")


In [3]:
ol.ip_dict

{'krnl_vadd_1': {'phys_addr': 2147483648,
  'addr_range': 60,
  'type': 'xilinx.com:hls:krnl_vadd:1.0',
  'hw_control_protocol': 'ap_ctrl_chain',
  'fullpath': 'krnl_vadd_1',
  'registers': {'CTRL': {'address_offset': 0,
    'access': 'read-write',
    'size': 4,
    'description': 'OpenCL Control Register',
    'type': 'unsigned int',
    'id': None,
    'fields': {'AP_START': {'access': 'read-write',
      'bit_offset': 0,
      'bit_width': 1,
      'description': 'Start the accelerator'},
     'AP_DONE': {'access': 'read-only',
      'bit_offset': 1,
      'bit_width': 1,
      'description': 'Accelerator has finished - cleared on read'},
     'AP_IDLE': {'access': 'read-only',
      'bit_offset': 2,
      'bit_width': 1,
      'description': 'Accelerator is idle'},
     'AP_READY': {'access': 'read-only',
      'bit_offset': 3,
      'bit_width': 1,
      'description': 'Accelerator is ready to start next computation'},
     'AUTO_RESTART': {'access': 'read-write',
      'bit_offs

In [4]:
vadd = ol.krnl_vadd_1

In [11]:
vadd?

Type:        DefaultIP
String form: <pynq.overlay.DefaultIP object at 0xaf40f598>
File:        /usr/local/share/pynq-venv/lib/python3.8/site-packages/pynq/overlay.py
Docstring:  
Driver for an IP without a more specific driver

This driver wraps an MMIO device and provides a base class
for more specific drivers written later. It also provides
access to GPIO outputs and interrupts inputs via attributes. More specific
drivers should inherit from `DefaultIP` and include a
`bindto` entry containing all of the IP that the driver
should bind to. Subclasses meeting these requirements will
automatically be registered.

Attributes
----------
mmio : pynq.MMIO
    Underlying MMIO driver for the device
_interrupts : dict
    Subset of the PL.interrupt_pins related to this IP
_gpio : dict
    Subset of the PL.gpio_dict related to this IP


In [5]:
# allocate buffers
size = 1024*1024
in1_vadd = pynq.allocate((1024, 1024), np.uint32)
in2_vadd = pynq.allocate((1024, 1024), np.uint32)
out = pynq.allocate((1024, 1024), np.uint32)



In [6]:
# initialize input
in1_vadd[:] = np.random.randint(low=0, high=100, size=(1024, 1024), dtype=np.uint32)
in2_vadd[:] = 200



In [7]:
# send data to the device
in1_vadd.sync_to_device()
in2_vadd.sync_to_device()
in1_vadd

PynqBuffer([[44, 56, 36, ..., 39, 56, 75],
            [30, 89, 23, ..., 14, 40, 94],
            [ 3, 93, 96, ..., 77, 34, 67],
            ...,
            [29, 50, 86, ..., 74,  6, 70],
            [62, 47, 64, ..., 68, 62, 34],
            [98, 87, 14, ..., 80, 27, 17]], dtype=uint32)

In [8]:

# call kernel
# vadd.call(in1_vadd, in2_vadd, out, size)

# get data from the device
out.sync_from_device()

# check results
msg = "SUCCESS!" if np.array_equal(out, in1_vadd + in2_vadd) else "FAILURE!"
print(msg)



FAILURE!


In [9]:
out

PynqBuffer([[0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            ...,
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0],
            [0, 0, 0, ..., 0, 0, 0]], dtype=uint32)

In [10]:
# clean up
# del in1_vadd
# del in2_vadd
# del out
# ol.free()